<a href="https://colab.research.google.com/github/young-hwanlee/autoencoder/blob/master/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/young-hwanlee/autoencoder.git

In [2]:
ls -ltr

total 8
drwxr-xr-x 1 root root 4096 Mar  5 14:37 sample_data/
drwxr-xr-x 3 root root 4096 Mar 16 05:13 autoencoder/


In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
#%%
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

from tqdm import trange
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data/")

ModuleNotFoundError: ignored

In [3]:
# mnist = tf.keras.datasets.mnist.load_data()

In [4]:
## Set up the seed for reproducibility.
seed = 42

# tf.reset_default_graph()          # It seems like it's only working in the old version.
# tf.set_random_seed(seed)          # It seems like it's only working in the old version.

tf.compat.v1.reset_default_graph()
tf.compat.v1.set_random_seed(seed)
np.random.seed(seed)

In [5]:
## Set up the hyperparameters
scale_weights = 1e-4
learning_rate = 1e-1
n_epochs = 50
batch_size = 100

In [23]:
#%%
## [Construction phase]
n_inputs = 28*28
n_hidden1 = 256
n_hidden2 = 256

n_codes = 100
n_outputs = n_inputs

n_train = 20000
# n_train = 55000   # over 60,000

In [19]:
type(mnist)

tuple

In [20]:
mnist

((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [22]:
len(mnist)

2

In [24]:
# X_train = mnist.train.images[:n_train]   # It seems like it's only working in the old version.
# X_val = mnist.validation.images          # It seems like it's only working in the old version.
# X_test = mnist.test.images               # It seems like it's only working in the old version.

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = tf.reshape(X_train , [-1 , 784])
X_test = tf.reshape(X_test , [-1 , 784])

In [25]:
# len(X_train[:n_train])
# len(X_train[n_train:])

X_train[:n_train].shape

TensorShape([Dimension(20000), Dimension(784)])

In [26]:
X_train[-5000:].shape

TensorShape([Dimension(5000), Dimension(784)])

In [27]:
X_train = X_train[:n_train]
X_val = X_train[-5000:]

# X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")   # It seems like it's only working in the old version.
# y = tf.placeholder(tf.float32, shape=[None, n_codes], name="y")    # It seems like it's only working in the old version.

tf.compat.v1.disable_eager_execution()      # In order to run the codes with TensorFlow 2.x
X = tf.compat.v1.placeholder(tf.float32, shape=[None, n_inputs], name="X")
y = tf.compat.v1.placeholder(tf.float32, shape=[None, n_codes], name="y")

In [28]:
b1 = tf.Variable(tf.zeros(shape=[n_hidden1]), dtype=tf.float32)
b2 = tf.Variable(tf.zeros(shape=[n_codes]), dtype=tf.float32)
b3 = tf.Variable(tf.zeros(shape=[n_hidden2]), dtype=tf.float32)
b4 = tf.Variable(tf.zeros(shape=[n_inputs]), dtype=tf.float32)

## He initialization (with any variant of ReLU)
initializer = tf.contrib.layers.variance_scaling_initializer(mode='FAN_AVG')
W1 = tf.Variable(initializer(shape=[n_inputs, n_hidden1]), dtype=tf.float32, name='W1')
W2 = tf.Variable(initializer(shape=[n_hidden1, n_codes]), dtype=tf.float32, name='W2')
W3 = tf.Variable(initializer(shape=[n_codes, n_hidden2]), dtype=tf.float32, name='W3')
W4 = tf.Variable(initializer(shape=[n_hidden2, n_outputs]), dtype=tf.float32, name='W4')

In [29]:
with tf.name_scope("encoder"):  # to group related nodes
    hidden1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    codes = tf.matmul(hidden1, W2) + b2
    codes_with_activation_fnc = tf.nn.relu(codes)

with tf.name_scope("decoder"):  # to group related nodes
    hidden2 = tf.nn.relu(tf.matmul(codes_with_activation_fnc, W3) + b3)
    X_prime = tf.matmul(hidden2, W4) + b4

with tf.name_scope("loss"):  # to group related nodes
    reg_loss = tf.reduce_sum(tf.square(W1)) + tf.reduce_sum(tf.square(W2)) \
               + tf.reduce_sum(tf.square(W3)) + tf.reduce_sum(tf.square(W4))
    loss = tf.add(tf.reduce_mean(tf.square(X - X_prime)), scale_weights * reg_loss, name="loss")

with tf.name_scope("train"):  # to group related nodes
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9, use_nesterov=True)
    training_op = optimizer.minimize(loss)

In [30]:
#%%
## [Excution phase]
init = tf.global_variables_initializer()
saver = tf.train.Saver()  # to save the trained model parameters to disk

# if cross_entropy_check == "1":
#     acc_train_hist, acc_val_hist = [], []
loss_train_hist, loss_val_hist = [], []

with tf.Session() as sess:
    init.run()

    for epoch in trange(n_epochs):
        for iteration in range(int(np.shape(X_train)[0] // batch_size)):
            X_batch = X_train[iteration * batch_size:(iteration + 1) * batch_size]
            sess.run(training_op, feed_dict={X: X_batch.eval()})    # Add ".eval()" to make it run

            loss_train = loss.eval(feed_dict={X: X_batch.eval()})
            loss_val = loss.eval(feed_dict={X: X_val.eval()})
            loss_train_hist.append(loss_train)
            loss_val_hist.append(loss_val)

            if iteration % 100 == 0:
                print("\nepoch :", epoch, "\titeration :", iteration)
                print("loss :", loss_val)

    X_prime_check = X_prime.eval(feed_dict={X: X_val.eval()})


  0%|          | 0/50 [00:00<?, ?it/s]


epoch : 0 	iteration : 0
loss : 2.1183099e+21

epoch : 0 	iteration : 100
loss : nan



  2%|▏         | 1/50 [02:47<2:16:57, 167.71s/it]


epoch : 1 	iteration : 0
loss : nan

epoch : 1 	iteration : 100
loss : nan



  4%|▍         | 2/50 [05:38<2:15:00, 168.76s/it]

...


  6%|▌         | 3/50 [08:49<2:17:13, 175.19s/it]


epoch : 3 	iteration : 0
loss : nan

epoch : 3 	iteration : 100
loss : nan



  8%|▊         | 4/50 [12:09<2:20:01, 182.65s/it]


epoch : 4 	iteration : 0
loss : nan

epoch : 4 	iteration : 100
loss : nan


KeyboardInterrupt: ignored

# ** It seems like the coding is not working well after the version of TensorFlow was upgraded. And I need to modify a lot.**

In [ ]:
#%%
## Check the results
print("MSE : ", np.sum(np.square(X_val - X_prime_check))/(np.shape(X_val)[0]*np.shape(X_val)[1]))

plt.figure()
plt.plot(loss_train_hist)
plt.plot(loss_val_hist)
plt.ylabel('loss')
plt.xlabel('iterations')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

#%%
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_val[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(X_prime_check[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

fig.text(0.475, 0.85, 'Original Figures', va='center')
fig.text(0.45, 0.425, 'Reconstructed Figures', va='center')
    
plt.show()